# Robotic Exploration Investigation (with Camera, Distance Sensor and Enhanced Analysis)

In this investigation you will configure the Jetbot to move forward and then spin in order to capture a series of images and distances from its environment. The distances will be used to generate a 2D polar grid of the readings. The images will be processed through a DepthNet Model and MobileNet model to derive corresponding depth maps and identify objects from the robot's environment. This set of data will help you to gather and analyze visual information from an environment where you may have limited or no visibility. 

## 1. Import Libraries

In this section, we import the necessary libraries. Next, we'll initialize the sensors and devices required for our analysis.

In [ ]:
import time
import threading
import uuid
import subprocess
from smbus2 import SMBus
from VL53L1X import VL53L1X
import cv2
import ipywidgets as widgets
import traitlets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg
import os
import shutil
import zipfile
import csv
import jetson_inference
import jetson_utils
import numpy as np
import matplotlib.pyplot as plt

## 2. Initialization

In this section, we initialize the distance sensor, camera, robot, and directory to store the data from our investigation. The ``range_value`` variable controls whether the distance sensor is calibrated for short, medium, or long distance ranges and may be modified based on your environment. The ``framerate`` variable controls how many distance sensor readings and camera frames are captured per second while the robot is collecting data.

In [ ]:
# Initialize VL53L1X distance sensor
# Set I2C bus number
bus_number = 0

#Set distance sensor range value
# 0 = Unchanged
# 1 = Short Range
# 2 = Medium Range
# 3 = Long Range
range_value = 3

tof = VL53L1X(i2c_bus=bus_number)
tof.open()

tof.start_ranging(range_value)

In [ ]:
# Initialize camera and robot
camera = Camera.instance(width=640, height=360)
robot = Robot()

framerate = 20

# Path to the snapshots directory
snapshots_dir = 'snapshots'
# Check if the snapshots directory exists
if os.path.exists(snapshots_dir):
    # Remove the directory and all its contents
    shutil.rmtree(snapshots_dir)
# Recreate the snapshots directory
os.makedirs(snapshots_dir)

## 3. Raw Data Visualization

Here, we set up the displays for the camera feed and distance sensor. Ensure that the camera and distance sensor are working and oriented to capture the robot's environment. 

In [ ]:
# Create cameara and distance widgets
image_widget = widgets.Image(format='jpeg', width=camera.width, height=camera.height)
distance_widget = widgets.Label(value=f"Distance: {tof.get_distance()} mm")
display(image_widget, distance_widget)

# Update display function
def display_distance():
    while True:
        distance_widget.value = f"Distance: {tof.get_distance()} mm"
        # Sleep for a short duration before the next update
        time.sleep(0.2)

# Start the update function in a separate thread
thread = threading.Thread(target=display_distance)
thread.start()

link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

## 4. Robot Movement Calibration

Movement on the Jetbot is controlled by modifying a combination of motor power and movement time. The surface that the Jetbot drives on can have a large impact on how far and fast it actually moves. This section progams your robot to turn. 

Use this section to calibrate the necessary motor power and movement time for your robot's environment. Place your robot in the environment where you plan to capture images and run the code block below. The goal is for the robot to complete a 360-degree turn as slowly as possible (allowing more data to be captured). Adjust the ``robot_speed`` and ``time_to_360`` variables until the robot is calibrated for these results.

In [ ]:
# Calibrate robot speed and duration for 360 degree turn
robot_speed = 0.09
time_to_360 = 6.0

robot.left(robot_speed)
time.sleep(time_to_360)  
robot.stop()

## 5. Raw Data Collection

This section directs the robot to drive forward and capture frames from the camera and distance sensor readings. The ``robot_speed`` and ``time_to_360`` variables set in the previous step are used here.  An additional ``time_forward`` variable here may be modified to control the duration that the robot drives forward before capturing data.

In [ ]:
# Variable to control how long the robot drives forward before collecting data
time_forward = 0

# Capture raw video frames and distances as robot spins
video_frames = []
distance_values = []
image_uuids = []

def capture_data(duration=time_to_360):
    start_time = time.time()
    next_capture_time = start_time + 1.0/framerate
    
    while time.time() - start_time < duration:
        current_time = time.time()
        if current_time >= next_capture_time:
            frame = camera.value
            
            # Extracting distance from the widget's value, assuming the format "Distance: XXXX mm"
            distance_str = distance_widget.value.split(":")[1].strip().split(" ")[0]
            distance = float(distance_str)

            # Generate a UUID and save it to the list
            frame_uuid = str(uuid.uuid1())
            image_uuids.append(frame_uuid)

            video_frames.append((frame_uuid, frame))
            distance_values.append(distance)
            
            # Adjust next capture time
            next_capture_time += 1.0/framerate
        else:
            # Sleep only the remaining time
            time.sleep(next_capture_time - current_time)

    # Save the buffered frames to disk after capturing
    for frame_uuid, frame in video_frames:
        file_path = os.path.join(snapshots_dir, frame_uuid + '.jpg')
        cv2.imwrite(file_path, frame)
        
#Drive robot forward desired duration
robot.forward(0.3)
time.sleep(time_forward)
robot.stop()
        
#Turn robot and capture data
robot.left(speed=robot_speed)
capture_data()
robot.stop()

## 6. Raw Data Display

This section sets up a display where the camera frames and corresponding distance values may be inspected. Use the scrubber to inspect the data collected.

In [ ]:
# Display raw video frames and distance values
def display_playback(video_frames, distance_values):
    # Create image and text widgets
    image_widget = widgets.Image(format='jpeg', width=camera.width, height=camera.height)
    distance_widget = widgets.Label(value=f"Distance: {distance_values[0]} mm")
    slider = widgets.IntSlider(value=0, min=0, max=len(video_frames)-1, step=1, description="Frame")
    
    # Update function for the widgets
    def update_widgets(change):
        image_widget.value = bgr8_to_jpeg(video_frames[slider.value][1])  # Access the frame from the tuple
        distance_widget.value = f"Distance: {distance_values[slider.value]} mm"
    
    # Observe the slider changes
    slider.observe(update_widgets, names='value')
    
    # Initial widget setup
    image_widget.value = bgr8_to_jpeg(video_frames[0][1])  # Access the frame from the tuple
    display(image_widget, slider, distance_widget)

# Use the function to display captured data
display_playback(video_frames, distance_values)

## 7. Camera and Distance Sensor Cleanup

This section stops the camera and distance sensor from continuing to run, freeing up resources for the remainder of the analysis.

In [ ]:
# Stop camera and distance sensor
camera.stop()
tof.stop_ranging()
tof.close()

## 8. Save Raw Video

This step generates a video file of the raw frames from the camera feed.

In [ ]:
# Save raw video data
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('raw_video.avi', fourcc, framerate, (camera.width, camera.height))

for frame_tuple in video_frames:
    actual_frame = frame_tuple[1]  # Extract the frame from the tuple
    out.write(actual_frame)

out.release()

## 9. Function Definition for 2D Polar Grid of Distance Readings

This section defines a function which generates a 2D polar grid of the distance sensor readings taken by the robot.

In [ ]:
# Function definition to display distance sensor readings in a 2D polar grid
def display_distance_values_polar_view(distance_values, max_distance=8000):
    # Convert distance_values to numpy array if it's not
    distance_values = np.array(distance_values)
    
    # Clip values and identify which are clipped
    clipped = (distance_values > max_distance) | (distance_values < 0)
    distance_values_clipped = np.minimum(distance_values, max_distance)
    
    # Generate theta values (rotate by 90 degrees, or pi/2 radians)
    theta = np.linspace(np.pi/2, 2*np.pi + np.pi/2, len(distance_values))
    
    # Convert distance readings to Cartesian coordinates
    x = distance_values_clipped * np.cos(theta)
    y = distance_values_clipped * np.sin(theta)
    
    # Create a scatter plot with polar grid
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.scatter(x[~clipped], y[~clipped], s=8, label='Valid Readings')  # Valid readings
    ax.scatter(x[clipped], y[clipped], s=8, color='red', label='Clipped Readings')  # Clipped readings
    
    # Setting the limits
    ax.set_xlim(-max_distance, max_distance)
    ax.set_ylim(-max_distance, max_distance)
    
    # Remove the x and y axis labels and ticks
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    ax.axhline(0, color='black',linewidth=0.5)
    ax.axvline(0, color='black',linewidth=0.5)
    
    # Adding polar grid
    ax.set_aspect('equal', adjustable='box')
    for angle in np.linspace(0, 2*np.pi, 12, endpoint=False):  # 12 lines for 30 degree intervals
        x_line = max_distance * np.cos(angle)
        y_line = max_distance * np.sin(angle)
        ax.plot([0, x_line], [0, y_line], color='gray', linestyle='--', linewidth=0.5)
    
    # Adding circular grid lines and their labels
    for r in np.linspace(0, max_distance, 5):  # 5 circles as grid lines
        circle = plt.Circle((0, 0), r, color='gray', fill=False, linestyle='--', linewidth=0.5)
        ax.add_artist(circle)
        # Add radial distance labels along one of the polar lines (choosing angle = pi/4 for placement)
        ax.text(r * np.cos(np.pi/4), r * np.sin(np.pi/4), str(int(r)), color='gray', ha='center', va='center')
    
    # Add degree markers for 0°, 90°, 180°, and 270°.
    ax.text(max_distance*0.01, max_distance*1.05, '0°', horizontalalignment='center')
    ax.text(max_distance*1.05, 0, '270°', verticalalignment='center')
    ax.text(-max_distance*0.01, -max_distance*1.1, '180°', horizontalalignment='center')
    ax.text(-max_distance*1.1, 0, '90°', verticalalignment='center')
    
    # Adding arrow for initial orientation (0° heading)
    arrow_length = max_distance * 0.0  # Length of the arrow for visibility
    ax.arrow(0, 0, 0, arrow_length, head_width=max_distance*0.05, head_length=max_distance*0.1, fc='green', ec='green')
    
    # Adding legend
    ax.legend()
    
    if os.path.exists('2D_plot.png'):
        os.remove('2D_plot.png')
    plt.savefig('2D_plot.png')
    plt.show()

## 10. Display 2D Polar Grid of Distance Readings

This section runs the function, displaying the 2D polar grid of distance sensor readings. A ``cutoff_threshold`` variable may be set to "clip" distance readings that are outside of a useable range. Distance values above the ``cutoff_threshold`` variable will be set equal to its value and displayed as a red dot on the polar grid. 

In [ ]:
cutoff_threshold = 800

# Plotting the distance values
display_distance_values_polar_view(distance_values, cutoff_threshold)

## 11. Image Analysis: Panorama

Now, let's stitch together a sample the raw camera frames that were taken into a panorama. An ``n`` variable allows you to specify the frequency of frrames used to generate the panorama.

This process may take some time to complete depending on the number of frames used, and may fail if the algorithm is unable to determine the overlapping portions from one image to another. If the process succeeds, a panoramic image will be displayed below. If the process fails, you may need to adjust the frequency of frames used or the speed of the robot.

In [ ]:
# Define value for 'n' to select every nth frame
n = 10 

# Select every nth frame tuple
selected_frame_tuples = video_frames[::n]

# Extract the actual frames from the selected tuples
imgs = [frame_tuple[1] for frame_tuple in selected_frame_tuples]

# Check if any image is empty or not read correctly
for idx, img in enumerate(imgs):
    if img is None:
        print(f"Image at index {idx} is empty or not read correctly. Please check the image path and format.")

# Stitch the images to create a panorama
stitcher = cv2.Stitcher_create()
(dummy, panorama) = stitcher.stitch(imgs)

if dummy != cv2.STITCHER_OK:
    print("Error in stitching images.")
else:
    # Check if "recent_panorama.jpg" exists and remove it
    if os.path.exists('camera_panorama.jpg'):
        os.remove('camera_panorama.jpg')
    
    # Save the panorama as "recent_panorama.jpg"
    cv2.imwrite('camera_panorama.jpg', panorama)

    # Display the panorama using IPython's display functionality
    from IPython.display import display, Image
    import PIL.Image

    # Convert OpenCV image to PIL image format for display
    if panorama is not None:
        panorama_pil = PIL.Image.fromarray(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
        display(panorama_pil)
    else:
        print("The panorama is empty. It seems the images couldn't be stitched together.")
        
del imgs

## 12. Load the DepthNet Model

DepthNet is designed to estimate depth from single RGB images. The "fcn-resnet18" variant in the notebook leverages a fully convolutional network (FCN) based on the ResNet-18 architecture. By processing RGB images, the model estimates depth information, which can be visualized as a depth map.

In [ ]:
# Load DepthNet Model

# Initialize the DepthNet model
net_depth = jetson_inference.depthNet("fcn-resnet18")
depth_field = net_depth.GetDepthField()

## 13. Process Depth Maps

In this step, the DepthNet model processes each raw video frame to generate a depth map with estimated minimum and maximum depth values.

In [ ]:
# Generate Depth Maps
depth_maps = []
depth_min_values = []  # List to store min depth values for each frame
depth_max_values = []  # List to store max depth values for each frame

# Process each video frame through DepthNet
def generate_depth_maps():
    for idx, (uuid, frame) in enumerate(video_frames):  # Unpack the tuple here
        cuda_img = jetson_utils.cudaFromNumpy(frame)
        net_depth.Process(cuda_img)
        depth_numpy = jetson_utils.cudaToNumpy(depth_field)
    
        # Check for NaN or Inf and replace them
        depth_numpy = np.nan_to_num(depth_numpy)
    
        # Ensure we aren't dividing by zero in normalization
        min_depth = np.min(depth_numpy)
        max_depth = np.max(depth_numpy)
        denom = max_depth - min_depth
        if denom == 0:
            denom = 1e-10  # Small constant to avoid division by zero
    
        # Normalize the depth values between 0 and 1
        normalized_depth = (depth_numpy - min_depth) / denom
    
        # Scale the normalized values between 0 and 255
        scaled_depth = (normalized_depth * 255).astype(np.uint8)
    
        # Apply the colormap
        depth_colormap = cv2.applyColorMap(scaled_depth, cv2.COLORMAP_JET)
    
        # Resize to match the original frames
        resized_depth_map = cv2.resize(depth_colormap, (camera.width, camera.height))
    
        # Use the UUID from the list to save the depth map
        depth_file_path = os.path.join(snapshots_dir, uuid + '_depthMap.jpg')
        cv2.imwrite(depth_file_path, resized_depth_map)
    
        # Append the min and max values to their respective lists
        depth_min_values.append(min_depth)
        depth_max_values.append(max_depth)
    
        depth_maps.append(resized_depth_map)

# Calling the function (for demonstration purposes)
generate_depth_maps()

## 14. Display Depth Maps

This section sets up a display where the raw camera frames and corresponding depth maps may be inspected. Use the scrubber to inspect the data analysis.

In [ ]:
# Display Depth Maps and Depth Estimates side-by-side with original frames

def display_frames_with_depthMaps(video_frames, depth_maps, depth_min_values, depth_max_values):
    # Create image widgets
    orig_image_widget = widgets.Image(format='jpeg', width=camera.width, height=camera.height)
    depth_image_widget = widgets.Image(format='jpeg', width=camera.width, height=camera.height)
    slider = widgets.IntSlider(value=0, min=0, max=len(video_frames)-1, step=1, description="Frame")
    
    # Create labels for min and max distances
    min_depth_label = widgets.Label(value=f"Min Depth: {depth_min_values[0]:.2f} units")
    max_depth_label = widgets.Label(value=f"Max Depth: {depth_max_values[0]:.2f} units")
    
    # Group min and max depth labels in an HBox
    depth_labels_hbox = widgets.HBox([min_depth_label, max_depth_label])
    
    # Group the depth image widget and the depth labels in a VBox
    depth_vbox = widgets.VBox([depth_image_widget, depth_labels_hbox])
    image_vbox = widgets.VBox([orig_image_widget, slider])
    
    # Update function for the widgets
    def update_widgets(change):
        orig_image_widget.value = bgr8_to_jpeg(video_frames[slider.value][1])  # Extract the frame from the tuple
        depth_image_widget.value = cv2.imencode('.jpg', depth_maps[slider.value])[1].tobytes()
        
        # Update min and max depth labels
        min_depth_label.value = f"Min Depth: {depth_min_values[slider.value]:.2f} units"
        max_depth_label.value = f"Max Depth: {depth_max_values[slider.value]:.2f} units"
    
    # Observe the slider changes
    slider.observe(update_widgets, names='value')
    
    # Initial widget setup
    orig_image_widget.value = bgr8_to_jpeg(video_frames[0][1])  # Extract the frame from the tuple
    depth_image_widget.value = cv2.imencode('.jpg', depth_maps[0])[1].tobytes()
    
    # Display the widgets using an outer HBox
    display(widgets.HBox([image_vbox, depth_vbox]))

# Call the modified function to display the frames
display_frames_with_depthMaps(video_frames, depth_maps, depth_min_values, depth_max_values)

## 15. Load the MobileNet Model

The SSD-MobileNet-v2 model performs object recognition. The ``threshold`` parameter specifies the confidence level below which detections will be discarded.

In [ ]:
# Initialize the MobileNet model
net_mobile = jetson_inference.detectNet("ssd-mobilenet-v2", threshold=0.3)

## 16. Perform Object Recognition

In this step, the MobileNet model processes each raw video frame to identify objects above the necessary confidence threshold.

In [ ]:
# Process frames for object detection
object_out = cv2.VideoWriter('object_detection_video.avi', cv2.VideoWriter_fourcc(*'XVID'), framerate, (camera.width, camera.height))

# Create image widget
object_detection_widget = widgets.Image(format='jpeg', width=camera.width, height=camera.height)

# Create a list to store the detection results for each frame
detection_results = []

# Enhanced object detection loop to store object names and their confidence scores
for (uuid, frame) in video_frames:  # Unpack the tuple here
    cuda_img = jetson_utils.cudaFromNumpy(frame)
    detections = net_mobile.Detect(cuda_img, overlay="box,labels,conf")
    
    # Storing detected object names and their confidence scores for the current frame
    frame_detections = [(det.ClassID, net_mobile.GetClassDesc(det.ClassID), det.Confidence) for det in detections]
    detection_results.append(frame_detections)
    
    processed_img = jetson_utils.cudaToNumpy(cuda_img)
    object_detection_widget.value = cv2.imencode('.jpg', processed_img)[1].tobytes()
    object_out.write(processed_img)
object_out.release()

## 17. Display Object Recognition

This section sets up a display where the camera frames overlayed with objects recognized with the MobileNet model may be inspected. Use the scrubber to inspect the data analysis.

In [ ]:
# Display the frames with the object detection
def display_object_detections(video_frames):
    # Create a slider for frame navigation
    slider = widgets.IntSlider(value=0, min=0, max=len(video_frames)-1, step=1, description="Frame")
    
    # Create a list to store processed frames (with object detections)
    processed_frames = []
    
    # Process video and save object detections
    for (uuid, frame) in video_frames:  # Unpack the tuple here
        cuda_img = jetson_utils.cudaFromNumpy(frame)
        detections = net_mobile.Detect(cuda_img, overlay="box,labels,conf")
        processed_img = jetson_utils.cudaToNumpy(cuda_img)
        processed_frames.append(processed_img)

    # Update function for the widget
    def update_frame(change):
        object_detection_widget.value = cv2.imencode('.jpg', processed_frames[slider.value])[1].tobytes()

    # Attach the update function to the slider
    slider.observe(update_frame, names='value')
    
    # Initial display setup
    object_detection_widget.value = cv2.imencode('.jpg', processed_frames[0])[1].tobytes()
    
    # Display the widget and slider
    display(object_detection_widget, slider)
    
display_object_detections(video_frames)

## 18. Generate Data Log

This section generates a CSV log file containing references to all raw camera frames, their corresponding distance sensor reading, and all corresponding data generated through the DepthNet and MobileNet analysis.

In [ ]:
# Generate CSV log of all data captured and analyzed
log_file_path = 'data_log.csv'
with open(log_file_path, 'w', newline='') as csvfile:
    log_writer = csv.writer(csvfile)
    
    # Write headers
    log_writer.writerow(["File Name", "Distance", "Depth Map Name", "Min Depth Value", "Max Depth Value", "Detected Objects and Confidence"])
    
    # Write data for each frame
    for idx, image_uuid in enumerate(image_uuids):
        # Creating a string representation for detected objects and their confidence scores
        objects_str = "; ".join([f"{det[1]} ({det[2]:.2f})" for det in detection_results[idx]])

        log_writer.writerow([
            f"{image_uuid}.jpg",
            distance_values[idx],
            f"{image_uuid}_depthMap.jpg",
            depth_min_values[idx],
            depth_max_values[idx],
            objects_str
        ])

## 19. Generate Data Archive

This section generates a ZIP file containing all captured and processed data generated through the investigation. It may be downloaded for further dissemination and archival. 

In [ ]:
# Function to zip the generated data
def zip_generated_files():
    # Specify the files and directories to be zipped
    files_to_zip = [
        'data_log.csv',  # The log file
        'raw_video.avi',
        'object_detection_video.avi',
        'camera_panorama.jpg',
        '2D_plot.png',
        snapshots_dir  # Directory containing original images and depth maps
        # Add other files like panorama, 2D plot, regular video, etc. if they are generated in the notebook
    ]
    
    # Create the zip file
    zip_filename = "data_analysis_files.zip"
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file_or_dir in files_to_zip:
            if os.path.isfile(file_or_dir):
                # Add individual file to the zip
                zipf.write(file_or_dir)
            elif os.path.isdir(file_or_dir):
                # Add directory and all its contents to the zip
                for foldername, subfolders, filenames in os.walk(file_or_dir):
                    for filename in filenames:
                        file_path = os.path.join(foldername, filename)
                        zipf.write(file_path, os.path.relpath(file_path, file_or_dir))
    
    return zip_filename  # Return the path to the generated zip file for verification

# Call the function to create the zip file
zip_generated_files()

### Conclusion

In this investigation, you successfully captured raw camera and distance sensor data from the robot's environment. You then performed a thorough analysis on the data, allowing you to gain insight into the robot's surroundings. 